In [1]:
# run first. then have fun.
from pyspark.sql.functions import col, to_date, to_timestamp
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, FloatType


## Generate the Schema (StructType) of the CSV Data (ecomm_behavior_data)

In [2]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, FloatType
schema = (StructType([
    StructField("event_time", StringType(), False),
    StructField("event_type", StringType(), False),
    StructField("product_id", IntegerType(), False),
    StructField("category_id", LongType(), False),
    StructField("category_code", StringType(), False),
    StructField("brand", StringType(), False),
    StructField("price", FloatType(), False),
    StructField("user_id", IntegerType(), False),
    StructField("user_session", StringType(), False),
]))

## Load the Dataset
> Note: The github repo contains the `-sm.csv` data sampled from [Kaggle: Ecommerce Behavior Data Multi Category Store](https://www.kaggle.com/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store). To follow along with the complete dataset, just download it and drop the `2019-Oct.csv, 2019-Nov.csv` files into the `datasets` directory in the project.
> `.load(f"{dataset_dir}/{datasets[0]}")` will load the month of October. Bump to `datasets[1]` to load the month of November.

In [3]:
dataset_dir = '/opt/spark/work-dir/hitchhikers_guide/datasets/ecomm_behavior_data'
# note: if you download the full dataset from https://www.kaggle.com/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store,
# just use the following and comment out the `-sm.csv` datasets.
# datasets = ['2019-Oct.csv','2019-Nov.csv']
datasets = ['2019-Oct-sm.csv','2019-Nov-sm.csv']

In [4]:
# read and process initial dataset
# 
ecomm_df = (
    spark.read.format("csv")
    .option("header", True)
    .schema(schema)
    .load(f"{dataset_dir}/{datasets[0]}") # loads '2019-10-01' data
    #.load(f"{dataset_dir}/{datasets[1]}")
)

In [5]:
ecomm_df.show(20, truncate=False)

+-----------------------+----------+----------+-------------------+-----------------------------------+--------+-------+---------+------------------------------------+
|event_time             |event_type|product_id|category_id        |category_code                      |brand   |price  |user_id  |user_session                        |
+-----------------------+----------+----------+-------------------+-----------------------------------+--------+-------+---------+------------------------------------+
|2019-10-01 00:00:00 UTC|view      |44600062  |2103807459595387724|NULL                               |shiseido|35.79  |541312140|72d76fde-8bb3-4e00-8c23-a032dfed738c|
|2019-10-01 00:00:00 UTC|view      |3900821   |2053013552326770905|appliances.environment.water_heater|aqua    |33.2   |554748717|9333dfbd-b87a-4708-9857-6336556b0fcc|
|2019-10-01 00:00:01 UTC|view      |17200506  |2053013559792632471|furniture.living_room.sofa         |NULL    |543.1  |519107250|566511c2-e2e3-422b-b695-cf8e6e

In [6]:
ecomm_df.count()

64

## Convert from CSV to Partitioned Parquet
While there is a simplicity to using CSV. It can be a problematic format to work with. Luckily, the ecomm dataset has already been preprocessed (cleaned).

**What we'll achieve**
1. We will do some minor post-processing, converting the `event_time` from a StringType to a DateTimeType. To do that we will be using the `to_timestamp` function. You'll notice that we need to `format` the timestamp conversion given we are parsing a string and need to reflect the format. `2019-10-01 00:00:00 UTC` is referenced using `yyyy-MM-dd HH:mm:ss z`.
2. Given the size of the data (~9GB for Nov, ~4GB for Oct) it also makes sense to partition by day (to speed up local processing). On that note, we also need to create a new column called `event_date` in order to store the partition information.

In [7]:
from pyspark.sql.functions import to_date, to_timestamp
spark.conf.set("spark.sql.parquet.compression.codec", "zstd")

sink_dir = 'sm' if datasets[1].endswith('-sm.csv') else 'lg'

(ecomm_df
   .withColumn("event_time", to_timestamp(col("event_time"), "yyyy-MM-dd HH:mm:ss z"))
   .withColumn("event_date", to_date(col("event_time")))
   .write
   .format("parquet")
   .partitionBy("event_date")
   .mode("append")
   .save(f"{dataset_dir}/parquet/{sink_dir}")
)

## Read Back our Parquet by Specific Date
> Note: Now that we have our schema (StructType) set, this will be encoded into the Parquet data. This simplifies reading back from our new parquet location (as long as we don't screw up or modify the schema since parquet doesn't have any notion of Schema Enforcement. This is a plus of working with Delta Lake, which we'll see in the rest of the Hitchhiker's Guide.

In [11]:
## Run a gutcheck on one of the days. See how things work. Probably pretty fast.
# if you've imported the `lg` data - switch 

#source_dir = 'sm' if datasets[0].endswith('-sm.csv') else 'lg'
source_dir = 'sm' if datasets[1].endswith('-sm.csv') else 'lg'
(spark.read
 .format("parquet")
 .load(f"{dataset_dir}/parquet/{source_dir}/")
 .where(col("event_date").eqNullSafe("2019-10-01"))
# .where(col("event_date").eqNullSafe("2019-10-01"))
 .show(10)
)

+-------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+----------+
|         event_time|event_type|product_id|        category_id|       category_code|   brand|  price|  user_id|        user_session|event_date|
+-------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+----------+
|2019-10-01 00:00:00|      view|  44600062|2103807459595387724|                NULL|shiseido|  35.79|541312140|72d76fde-8bb3-4e0...|2019-10-01|
|2019-10-01 00:00:00|      view|   3900821|2053013552326770905|appliances.enviro...|    aqua|   33.2|554748717|9333dfbd-b87a-470...|2019-10-01|
|2019-10-01 00:00:01|      view|  17200506|2053013559792632471|furniture.living_...|    NULL|  543.1|519107250|566511c2-e2e3-422...|2019-10-01|
|2019-10-01 00:00:01|      view|   1307067|2053013558920217191|  computers.notebook|  lenovo| 251.74|550050854|7c90fc70-0e80-459...|2019

## Where to go Next?
> This notebook only exists to help read, post-process, and write partitioned data from the ecomm dataset. We will be using the `parquet` data for the actual hitchhiker's guide. 

Now that we've learned how to read and write our csv data, it is time to tackle problems using Delta Lake. If you are new to using Delta Lake, then it is easiest to head over to [First Steps](../101-first-steps/README.md) to learn how to create and modify Delta Lake tables (with the intention of going from zero-to-hero).